In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-06 01:38:08.306358: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
  "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=700)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("lenet_3_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("lenet_3_pred.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                

Epoch 21/1000
5/5 [==============================] - 0s 23ms/step - loss: 16.3210 - mean_squared_error: 16.3210 - rmse: 4.0399 - mean_absolute_error: 3.1694 - val_loss: 15.8839 - val_mean_squared_error: 15.8839 - val_rmse: 3.9855 - val_mean_absolute_error: 2.9157 - lr: 0.0010
Epoch 22/1000
5/5 [==============================] - 0s 24ms/step - loss: 16.8139 - mean_squared_error: 16.8139 - rmse: 4.1005 - mean_absolute_error: 3.2023 - val_loss: 20.3200 - val_mean_squared_error: 20.3200 - val_rmse: 4.5078 - val_mean_absolute_error: 3.3052 - lr: 0.0010
Epoch 23/1000
5/5 [==============================] - 0s 21ms/step - loss: 15.9387 - mean_squared_error: 15.9387 - rmse: 3.9923 - mean_absolute_error: 3.1244 - val_loss: 13.8913 - val_mean_squared_error: 13.8913 - val_rmse: 3.7271 - val_mean_absolute_error: 2.8054 - lr: 0.0010
Epoch 24/1000
5/5 [==============================] - 0s 21ms/step - loss: 16.4052 - mean_squared_error: 16.4052 - rmse: 4.0503 - mean_absolute_error: 3.2355 - val_loss: 

Epoch 49/1000
5/5 [==============================] - 0s 27ms/step - loss: 14.8808 - mean_squared_error: 14.8808 - rmse: 3.8576 - mean_absolute_error: 3.0256 - val_loss: 14.9844 - val_mean_squared_error: 14.9844 - val_rmse: 3.8710 - val_mean_absolute_error: 2.8413 - lr: 2.5000e-04
Epoch 50/1000
5/5 [==============================] - 0s 32ms/step - loss: 14.0792 - mean_squared_error: 14.0792 - rmse: 3.7522 - mean_absolute_error: 3.0076 - val_loss: 15.7614 - val_mean_squared_error: 15.7614 - val_rmse: 3.9701 - val_mean_absolute_error: 2.8914 - lr: 2.5000e-04
Epoch 51/1000
5/5 [==============================] - 0s 26ms/step - loss: 14.3875 - mean_squared_error: 14.3875 - rmse: 3.7931 - mean_absolute_error: 3.0166 - val_loss: 16.4880 - val_mean_squared_error: 16.4880 - val_rmse: 4.0605 - val_mean_absolute_error: 2.9463 - lr: 2.5000e-04
Epoch 52/1000
5/5 [==============================] - 0s 30ms/step - loss: 14.2574 - mean_squared_error: 14.2574 - rmse: 3.7759 - mean_absolute_error: 2.9644 

5/5 [==============================] - 0s 20ms/step - loss: 14.8700 - mean_squared_error: 14.8700 - rmse: 3.8562 - mean_absolute_error: 2.9735 - val_loss: 15.5663 - val_mean_squared_error: 15.5663 - val_rmse: 3.9454 - val_mean_absolute_error: 2.8689 - lr: 1.2500e-04
Epoch 78/1000
5/5 [==============================] - 0s 21ms/step - loss: 12.5589 - mean_squared_error: 12.5589 - rmse: 3.5439 - mean_absolute_error: 2.7803 - val_loss: 14.3871 - val_mean_squared_error: 14.3871 - val_rmse: 3.7930 - val_mean_absolute_error: 2.7889 - lr: 1.2500e-04
Epoch 79/1000
5/5 [==============================] - 0s 21ms/step - loss: 12.9478 - mean_squared_error: 12.9478 - rmse: 3.5983 - mean_absolute_error: 2.8372 - val_loss: 14.5875 - val_mean_squared_error: 14.5875 - val_rmse: 3.8194 - val_mean_absolute_error: 2.8017 - lr: 1.2500e-04
Epoch 80/1000
5/5 [==============================] - 0s 23ms/step - loss: 13.6290 - mean_squared_error: 13.6290 - rmse: 3.6917 - mean_absolute_error: 2.8855 - val_loss: 15

5/5 [==============================] - ETA: 0s - loss: 14.0469 - mean_squared_error: 14.0469 - rmse: 3.7479 - mean_absolute_error: 2.9031
Epoch 00105: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
5/5 [==============================] - 0s 19ms/step - loss: 14.0469 - mean_squared_error: 14.0469 - rmse: 3.7479 - mean_absolute_error: 2.9031 - val_loss: 15.2830 - val_mean_squared_error: 15.2830 - val_rmse: 3.9093 - val_mean_absolute_error: 2.8457 - lr: 6.2500e-05
run:  1
pooling layers: 4
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 6)            1302      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 50, 6)            0         
 1D)      

...

Epoch 140/1000
5/5 [==============================] - 0s 17ms/step - loss: 10.0967 - mean_squared_error: 10.0967 - rmse: 3.1775 - mean_absolute_error: 2.4853 - val_loss: 8.8954 - val_mean_squared_error: 8.8954 - val_rmse: 2.9825 - val_mean_absolute_error: 2.2744 - lr: 5.0000e-04
Epoch 141/1000
5/5 [==============================] - 0s 17ms/step - loss: 10.4596 - mean_squared_error: 10.4596 - rmse: 3.2341 - mean_absolute_error: 2.5523 - val_loss: 9.3009 - val_mean_squared_error: 9.3009 - val_rmse: 3.0497 - val_mean_absolute_error: 2.2790 - lr: 5.0000e-04
Epoch 142/1000
5/5 [==============================] - 0s 20ms/step - loss: 10.3262 - mean_squared_error: 10.3262 - rmse: 3.2134 - mean_absolute_error: 2.4730 - val_loss: 8.6083 - val_mean_squared_error: 8.6083 - val_rmse: 2.9340 - val_mean_absolute_error: 2.2512 - lr: 5.0000e-04
Epoch 143/1000
5/5 [==============================] - 0s 17ms/step - loss: 10.6517 - mean_squared_error: 10.6517 - rmse: 3.2637 - mean_absolute_error: 2.5518 - 

5/5 [==============================] - 0s 17ms/step - loss: 10.3403 - mean_squared_error: 10.3403 - rmse: 3.2156 - mean_absolute_error: 2.4592 - val_loss: 9.0736 - val_mean_squared_error: 9.0736 - val_rmse: 3.0122 - val_mean_absolute_error: 2.2552 - lr: 5.0000e-04
Epoch 170/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.5778 - mean_squared_error: 9.5778 - rmse: 3.0948 - mean_absolute_error: 2.4034 - val_loss: 8.4924 - val_mean_squared_error: 8.4924 - val_rmse: 2.9142 - val_mean_absolute_error: 2.2268 - lr: 5.0000e-04
Epoch 171/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.3914 - mean_squared_error: 9.3914 - rmse: 3.0645 - mean_absolute_error: 2.4026 - val_loss: 8.7248 - val_mean_squared_error: 8.7248 - val_rmse: 2.9538 - val_mean_absolute_error: 2.2553 - lr: 5.0000e-04
Epoch 172/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.2720 - mean_squared_error: 9.2720 - rmse: 3.0450 - mean_absolute_error: 2.3885 - val_loss: 9.1999 - va

Epoch 198/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.3188 - mean_squared_error: 9.3188 - rmse: 3.0527 - mean_absolute_error: 2.3904 - val_loss: 8.6367 - val_mean_squared_error: 8.6367 - val_rmse: 2.9388 - val_mean_absolute_error: 2.2222 - lr: 2.5000e-04
Epoch 199/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.7743 - mean_squared_error: 9.7743 - rmse: 3.1264 - mean_absolute_error: 2.4534 - val_loss: 8.8234 - val_mean_squared_error: 8.8234 - val_rmse: 2.9704 - val_mean_absolute_error: 2.2172 - lr: 2.5000e-04
Epoch 200/1000
5/5 [==============================] - ETA: 0s - loss: 8.7842 - mean_squared_error: 8.7842 - rmse: 2.9638 - mean_absolute_error: 2.3056
Epoch 00200: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
5/5 [==============================] - 0s 18ms/step - loss: 8.7842 - mean_squared_error: 8.7842 - rmse: 2.9638 - mean_absolute_error: 2.3056 - val_loss: 8.6707 - val_mean_squared_error: 8.6707 - val_rmse: 2.9446 -

Epoch 226/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.5469 - mean_squared_error: 9.5469 - rmse: 3.0898 - mean_absolute_error: 2.3862 - val_loss: 8.6305 - val_mean_squared_error: 8.6305 - val_rmse: 2.9378 - val_mean_absolute_error: 2.2055 - lr: 6.2500e-05
Epoch 227/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.8533 - mean_squared_error: 9.8533 - rmse: 3.1390 - mean_absolute_error: 2.4166 - val_loss: 8.6857 - val_mean_squared_error: 8.6857 - val_rmse: 2.9471 - val_mean_absolute_error: 2.1995 - lr: 6.2500e-05
Epoch 228/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.2362 - mean_squared_error: 9.2362 - rmse: 3.0391 - mean_absolute_error: 2.3492 - val_loss: 8.6694 - val_mean_squared_error: 8.6694 - val_rmse: 2.9444 - val_mean_absolute_error: 2.1973 - lr: 6.2500e-05
Epoch 229/1000
5/5 [==============================] - 0s 17ms/step - loss: 8.7730 - mean_squared_error: 8.7730 - rmse: 2.9619 - mean_absolute_error: 2.2331 - val_loss

Epoch 255/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.7651 - mean_squared_error: 9.7651 - rmse: 3.1249 - mean_absolute_error: 2.4331 - val_loss: 8.5379 - val_mean_squared_error: 8.5379 - val_rmse: 2.9220 - val_mean_absolute_error: 2.1838 - lr: 3.1250e-05
Epoch 256/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.4346 - mean_squared_error: 9.4346 - rmse: 3.0716 - mean_absolute_error: 2.3696 - val_loss: 8.5832 - val_mean_squared_error: 8.5832 - val_rmse: 2.9297 - val_mean_absolute_error: 2.1866 - lr: 3.1250e-05
Epoch 257/1000
5/5 [==============================] - 0s 17ms/step - loss: 8.6811 - mean_squared_error: 8.6811 - rmse: 2.9464 - mean_absolute_error: 2.2826 - val_loss: 8.5962 - val_mean_squared_error: 8.5962 - val_rmse: 2.9319 - val_mean_absolute_error: 2.1884 - lr: 3.1250e-05
Epoch 258/1000
5/5 [==============================] - 0s 17ms/step - loss: 9.0791 - mean_squared_error: 9.0791 - rmse: 3.0132 - mean_absolute_error: 2.3571 - val_loss

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)